In [193]:
import pandas as pd

import rpy2
import rpy2.robjects.packages as rpackages
from rpy2.robjects.packages import importr, data
import rpy2.robjects as robjects
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri

In [194]:
utils = importr('utils')
base = importr("base")
datasets = importr('datasets')
ggseg = importr("ggseg")
ggplot2 = importr("ggplot2")
dplyr = importr("dplyr")
tidyr = importr("tidyr")

ggseg3d = importr('ggseg3d')

In [195]:
utils.chooseCRANmirror(ind=1)
# utils.install_packages('NMF') # repos="https://cloud.r-project.org"

<rpy2.rinterface_lib.sexp.NULLType object at 0x7fc3713c77c0> [RTYPES.NILSXP]

### Convert PandasDataFrame to rpy2 DataFrame

In [196]:
dk_data = pd.read_csv("dk_R_subtype0.csv") 
# dk_data

aseg_data = pd.read_csv("aseg_R_subtype0.csv") 
# aseg_data

In [197]:
# dk_data_right = dk_data[dk_data['hemi']=='right']
# dk_data_left = dk_data[dk_data['hemi']=='left']

In [198]:
with localconverter(robjects.default_converter + pandas2ri.converter):
    dk_data_R = robjects.conversion.py2rpy(dk_data)

# with localconverter(robjects.default_converter + pandas2ri.converter):
#     dk_data_right_R = robjects.conversion.py2rpy(dk_data_right)

# with localconverter(robjects.default_converter + pandas2ri.converter):
#     dk_data_left_R = robjects.conversion.py2rpy(dk_data_left)

In [199]:
with localconverter(robjects.default_converter + pandas2ri.converter):
    aseg_data_R = robjects.conversion.py2rpy(aseg_data)

aseg_data_R

region,p
...,...


### Function description

In [200]:
print(ggseg3d.ggseg3d)

function (.data = NULL, atlas = "dk_3d", surface = "LCBC", hemisphere = c("right", 
    "subcort"), label = "region", text = NULL, colour = "colour", 
    palette = NULL, na.colour = "darkgrey", na.alpha = 1, show.legend = TRUE, 
    options.legend = NULL, ...) 
{
    atlas3d = get_atlas(atlas, surface = surface, hemisphere = hemisphere)
    if (!is.null(.data)) {
        atlas3d <- data_merge(.data, atlas3d)
    }
    pal.colours <- get_palette(palette)
    if (is.numeric(unlist(atlas3d[, colour]))) {
        if (is.null(names(palette))) {
            pal.colours$values <- seq(min(atlas3d[, colour], 
                na.rm = TRUE), max(atlas3d[, colour], na.rm = TRUE), 
                length.out = nrow(pal.colours))
        }
        atlas3d$new_col = gradient_n_pal(pal.colours$orig, pal.colours$values, 
            "Lab")(unlist(atlas3d[, colour]))
        fill = "new_col"
    }
    else {
        fill = colour
    }
    p = plotly::plot_ly()
    for (tt in 1:nrow(atlas3d)) {
       

In [201]:
print(ggseg3d.pan_camera)

function (p, camera, aspectratio = 1) 
{
    stopifnot(is.character(camera) | is.list(camera))
    views = if (class(camera) != "list") {
        camera <- match.arg(camera, c("left lateral", "left medial", 
            "right lateral", "right medial"))
        switch(camera, `left lateral` = list(eye = list(x = -2.5, 
            y = 0, z = 0)), `left medial` = list(eye = list(x = 2, 
            y = 0, z = 0)), `right lateral` = list(eye = list(x = 2, 
            y = 0, z = 0)), `right medial` = list(eye = list(x = -2.5, 
            y = 0, z = 0)))
    }
    else {
        camera
    }
    plotly::layout(p, scene = list(camera = views, aspectratio = aspectratio))
}
<bytecode: 0x7fc34fc95d98>
<environment: namespace:ggseg3d>



In [202]:
print(ggseg3d.get_palette)

function (palette) 
{
    if (is.null(palette)) {
        palette = c("skyblue", "dodgerblue")
    }
    if (!is.null(names(palette))) {
        pal.colours <- names(palette)
        pal.values <- unname(palette)
        pal.norm <- range_norm(pal.values)
    }
    else {
        pal.colours <- palette
        pal.norm <- seq(0, 1, length.out = length(pal.colours))
        pal.values <- seq(0, 1, length.out = length(pal.colours))
    }
    pal.colours = if (length(palette) == 1) {
        data.frame(values = c(pal.values, pal.values + 1), norm = c(0, 
            1), orig = c(pal.colours, pal.colours), stringsAsFactors = F)
    }
    else {
        data.frame(values = pal.values, norm = pal.norm, orig = pal.colours, 
            stringsAsFactors = F)
    }
    pal.colours$hex <- gradient_n_pal(colours = pal.colours$orig, 
        values = pal.colours$values, space = "Lab")(pal.colours$values)
    pal.colours
}
<bytecode: 0x7fc34fcf63b0>
<environment: namespace:ggseg3d>



In [203]:
print(ggseg3d.add_glassbrain)

function (p, hemisphere = c("left", "right"), colour = "#cecece", 
    opacity = 0.3) 
{
    cortex <- dplyr::filter(cortex_3d, hemi %in% hemisphere)
    cortex <- tidyr::unnest(cortex, ggseg_3d)
    colour <- if (grepl("^#", colour)) {
        colour
    }
    else {
        col2hex(colour)
    }
    for (tt in 1:nrow(cortex)) {
        col = rep(colour, length(cortex$mesh[[tt]]$it[1, ]))
        p = plotly::add_trace(p, x = cortex$mesh[[tt]]$vb["xpts", 
            ], y = cortex$mesh[[tt]]$vb["ypts", ], z = cortex$mesh[[tt]]$vb["zpts", 
            ], i = cortex$mesh[[tt]]$it[1, ] - 1, j = cortex$mesh[[tt]]$it[2, 
            ] - 1, k = cortex$mesh[[tt]]$it[3, ] - 1, facecolor = col, 
            type = "mesh3d", showscale = FALSE, name = "cerebral cortex", 
            opacity = opacity)
    }
    p
}
<bytecode: 0x7fc34fd72d88>
<environment: namespace:ggseg3d>



### Get atlases

In [204]:
dk_3d = ggseg3d.get_atlas('dk_3d', surface = ["LCBC",'inflated'], hemisphere = ['left','right'])
# dk_3d

In [205]:
aseg_3d = ggseg3d.get_atlas('aseg_3d', surface = "LCBC", hemisphere = 'subcort')
# aseg_3d

### Colors

In [206]:
# pallette = ggseg3d.get_palette(colors)

colors = robjects.r('''
c('#6e0101','#ffabab')
''')

print(colors)

[1] "#6e0101" "#ffabab"



In [207]:
# vminmax = [0,25]

colorbar = robjects.r('''
c(0,25)
''')

print(colorbar)

[1]  0 25



### Test ggseg3d function

In [210]:
plot_dk = ggseg3d.ggseg3d(dk_data_R, atlas = dk_3d, surface = ["LCBC",'inflated'], hemisphere = ['left','right'],
               label = 'region', colour = 'p', palette = colors, vminmax = colorbar)

plot_dk

<rpy2.robjects.vectors.ListVector object at 0x7fc3554360c0> [RTYPES.VECSXP]
R classes: ('plotly', 'htmlwidget')
[Lis..., NUL..., NUL..., Lis..., Lis..., NUL..., Sex..., Lis...]
  x: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7fc35308e300> [RTYPES.VECSXP]
  width: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7fc3713c77c0> [RTYPES.NILSXP]
  height: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7fc3713c77c0> [RTYPES.NILSXP]
  sizingPolicy: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7fc35308e300> [RTYPES.VECSXP]
  dependencies: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7fc35308e500> [RTYPES.VECSXP]
  elementId: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7fc3713c77c0> [RTYPES.NILSXP]
  preRenderHook: <class 'rpy2.rinterface.SexpClosure'>
  <rpy2.rinterface.SexpClosure object at 0x7fc35308e500> [RTYPES.CLOSXP]
  jsHooks: <class 'rpy2.rinterface.ListSexpVector'>
  <rpy2.rinterface.ListSexpVector object at 0x7fc35308e240> [RTYPES.VECSXP]

In [168]:
plot_aseg = ggseg3d.ggseg3d(aseg_data_R, atlas = aseg_3d, surface = "LCBC", hemisphere = 'subcort',
               label = 'region', colour = 'p', palette = colors, vminmax = [0,25])

print(plot_aseg)

In [209]:
type(plot_dk)

rpy2.robjects.vectors.ListVector

## DK-ATLAS --> Python

In [186]:
# LEFT HEMISPHERE

dk_left = ggseg3d.ggseg3d(dk_data_R, atlas = dk_3d, surface = ["LCBC",'inflated'], hemisphere = 'left',
               label = 'region', colour = 'p', palette = colors)

dk_left = ggseg3d.pan_camera(dk_left, 'right medial')


print(dk_left)

In [190]:
# RIGHT HEMISPHERE

dk_right = ggseg3d.ggseg3d(dk_data_R, atlas = dk_3d, surface = ["LCBC",'inflated'], hemisphere = 'right',
               label = 'region', colour = 'p', palette = colors)

dk_right = ggseg3d.pan_camera(dk_right, 'left medial')


print(dk_right)

In [188]:
# WHOLE BRAIN

dk_full = ggseg3d.ggseg3d(dk_data_R, atlas = dk_3d, surface = ["LCBC",'inflated'], hemisphere = ['left','right'],
               label = 'region', colour = 'p', palette = colors)

print(dk_full)

## ASEG-ATLAS --> Python

In [191]:
# WITHOUT REFERENCE

plot_aseg = ggseg3d.ggseg3d(aseg_data_R, atlas = aseg_3d, surface = "LCBC", hemisphere = 'subcort',
               label = 'region', colour = 'p', palette = colors)

print(plot_aseg)



In [177]:
# ADD GLASSBRAIN

plot_aseg = ggseg3d.add_glassbrain(plot_aseg, hemisphere = ["left", "right"], colour = "#cecece", opacity = 0.3)

print(plot_aseg)